In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from os import name
from google.colab import drive

# ML

## Modelo de Recomendacion de Machine Learning por Cosine Similarity


MONTAJE DE ARCHIVOS GOOGLE DRIVE:

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
user_reviews_content = '/content/drive/MyDrive/Colab Notebooks/HotelWiseML/Hoteles.NLP.01.NLTK.parquet'
user_reviews_dataset = pd.read_parquet(user_reviews_content)

MONTAJE DE ARCHIVOS LOCAL:

In [ ]:
user_reviews_content = 'Hoteles.NLP.01.NLTK.parquet'
user_reviews_dataset = pd.read_parquet(user_reviews_content)

## Funciones de Recomendación

**A continuación se probarán dos formas de calcular la similaridad del coseno**

### Función Nº1

In [80]:
def recomendacion_hotel(city: str) -> list:
    user_reviews_city = user_reviews_dataset[user_reviews_dataset['city'] == city]
    features = user_reviews_city[['security', 'avg_rating', 'sentiment_analysis']].copy()
    similarity_matrix = cosine_similarity(features)
    total_similarity = similarity_matrix.sum(axis=1)
    user_reviews_city_copy = user_reviews_city.copy()
    user_reviews_city_copy['total_similarity'] = total_similarity
    hoteles_sorted = user_reviews_city_copy.sort_values(by='total_similarity', ascending=True)
    return hoteles_sorted['name'].head(3)
print(recomendacion_hotel('Miami Beach'))

2        Best Western Plus Atlantik
1                     5 Star Island
5    Chic Apartments at Miami Beach
Name: name, dtype: object


### Función Nº2

In [81]:
def recomendacion_hotel1(city: str) -> list:
    user_reviews_city = user_reviews_dataset[user_reviews_dataset['city'] == city]
    hotel_item_matrix = user_reviews_city.pivot_table(index='city', columns=['name','security', 'avg_rating'], values='sentiment_analysis', aggfunc='mean', fill_value=0)
    city_row = hotel_item_matrix.loc[city].values.reshape(1, -1)
    similarity_scores = cosine_similarity(city_row, hotel_item_matrix.values)
    similar_users_indices = similarity_scores.argsort()[0][-6:-1]
    recommended_items = hotel_item_matrix.iloc[similar_users_indices].sum().sort_values(ascending=False).index.tolist()[:3]
    hotel_names = [f"Hotel recomendado {i+1}: {name}" for i, (name, _, _) in enumerate(recommended_items)]
    for hotel in hotel_names:
      print(hotel)
    return

recomendacion_hotel1('Miami Beach')

Hotel recomendado 1: 5 Star Island
Hotel recomendado 2: Best Western Plus Atlantik
Hotel recomendado 3: Chic Apartments at Miami Beach
